Load Model

In [ ]:
!pip install transformers accelerate sentencepiece flask pyngrok


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-DPO"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Document Generation

In [ ]:
SYSTEM_MSG_CON = """Ești un asistent care redactează contracte clare, profesioniste și bine structurate în limba română, respectând cerințele legale privind forma, conținutul și terminologia juridică."""

TEMPLATE_CON = """
Redactează un contract complet în limba română, pe baza următoarelor elemente:
- TipContract: {tip}
- Părțile contractante: {parti}
- Obiectul contractului: {obiect}
- Context/Scop: {scop}
- Obligațiile părților: {obligatii}
- Durată/Termen: {durata}
- Preț/Remunerație (dacă este cazul): {pret}
- Clauze speciale (penalități, confidențialitate etc.): {clauze}

Structura contractului trebuie să includă:
- Titlu
- Preambul
- Definiții (dacă este necesar)
- Corpul contractului cu articole și alineate numerotate
- Clauze finale
- Date de semnare
- Semnături
- Ștampile (dacă este cazul)

Asigură conformitatea cu cerințele legale pentru contractul respectiv. Adopta un ton {ton}. Nu genera nimic în afara contractului.
"""


def build_prompt_contracte(meta):
  print("entered build prompt")
  prompt = TEMPLATE_CON.format(
        tip=meta["TipContract"],
        parti=meta["PartiContractante"],
        obiect=meta["Obiect"],
        scop=meta["Context"],
        obligatii=meta["Obligatii"],
        durata=meta["Durata"],
        pret=meta["Pret"],
        clauze=meta["ClauzeSpeciale"],
        ton="Strict formal"
    )
  return prompt

In [ ]:
SYSTEM_MSG_REQ = """Ești un asistent care redactează contracte clare, politicoase și bine structurate în limba română, respectând toate cerințele formale impuse de lege."""

TEMPLATE_REQ = """
Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: {tip}
- Parti implicate: {parti}
- Context/Scop: {scop}
- Conflict juridic: {conflict}
- Argumente și probe: {argumente}
- Termen dorit: {termen}

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice. Adopta un ton {ton}, nu depasi 220 de cuvinte. Nu genera nimic in afara cererii.
"""

def build_prompt_req(meta):
  print("entered build prompt")
  prompt = TEMPLATE_REQ.format(
        tip=meta["TipCerere"],
        parti=meta["PartiImplicate"],
        scop=meta["Context"],
        conflict=meta["Conflict"],
        argumente=meta["Argumente"],
        termen=meta["Termen"],
        ton="Strict formal"
    )
  return prompt

In [ ]:
SYSTEM_MSG_PROC = """Ești un asistent care redactează procuri clare, juridic corecte și bine structurate în limba română, respectând cerințele legale privind forma, conținutul și terminologia specifică împuternicirilor și mandatelor."""
TEMPLATE_PROC = """
Redactează o procură completă în limba română, pe baza următoarelor elemente:
- TipProcura: {tip}
- Mandant: {mandant}
- Mandatar: {mandatar}
- Obiectul procurii / Împuternicirea: {obiect}
- Scop / Context: {scop}
- Puteri acordate: {puteri}
- Durata / Valabilitate: {durata}
- Restricții sau clauze speciale (dacă există): {clauze}

Structura procurii trebuie să includă:
- Titlu
- Preambul cu identificarea completă a părților
- Corpul documentului cu articole și alineate numerotate (cu atribuțiile exacte)
- Clauze privind responsabilitatea, limitele mandatului și revocarea
- Dată și locul întocmirii
- Semnătura mandantului
- Semnătura martorilor (dacă este cazul)
- Mențiuni privind autentificarea (dacă este cazul)

Adoptă un ton {ton}. Nu genera nimic în afara procurii.
"""
def build_prompt_proc(meta):
    prompt = TEMPLATE_PROC.format(
        tip=meta["TipProcura"],
        mandant=meta["Mandant"],
        mandatar=meta["Mandatar"],
        obiect=meta["Obiect"],
        scop=meta["Context"],
        puteri=meta["Puterile"],
        durata=meta["Durata"],
        clauze=meta["ClauzeSpeciale"],
        ton="Strict formal"
    )
    return prompt



In [ ]:
def call_model(system, prompt, temperature=0.4, max_new_tokens=600):

    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # extrage DOAR textul generat
    generated_only = decoded[len(full_prompt):].strip()

    # curăță \n\n brute → newline reale
    generated_only = generated_only.replace("\\n", "\n")

    del inputs, outputs
    generated_only=clean_text(generated_only)
    return generated_only

In [ ]:
import re

def clean_text(text, mode="document generation"):
    """
    Returnează doar textul până la ultima propoziție completă.
    Considerăm că o propoziție completă se termină cu '.', '!' sau '?'.
    """
    # Căutăm toate propozițiile care se termină corect
    if mode=="document_generation":
      sentences = re.findall(r'[^.!?\]]*[.!?\]]', text)
    else:
      sentences = re.findall(r'[^.!?]*[.!?]', text)

    if sentences:
        # Returnăm tot ce este până la ultima propoziție completă
        return ' '.join(sentences).strip()
    else:
        # Dacă nu s-a găsit nicio propoziție completă, returnăm textul original
        return text.strip()


In [ ]:
def generate_text(meta):
    device = "cuda" if torch.cuda.is_available() else "cpu"


    if "TipContract" in meta:
        system_msg = SYSTEM_MSG_CON
        prompt = build_prompt_contracte(meta)
        max_tokens=800
        call_fn = call_model

    elif "TipCerere" in meta:
        system_msg = SYSTEM_MSG_REQ
        prompt = build_prompt_req(meta)
        max_tokens=600
        call_fn = call_model

    elif "TipProcura" in meta:
        system_msg = SYSTEM_MSG_PROC
        prompt = build_prompt_proc(meta)
        max_tokens=600
        call_fn = call_model

    else:
        raise ValueError(
            "Meta trebuie să conțină TipContract, TipCerere sau TipProcura."
        )

    # 2. Generăm textul final
    print(prompt)
    text = call_fn(system_msg, prompt,temperature=0.4, max_new_tokens=max_tokens)
    text=clean_text(text)
    return text


In [ ]:
!pip install python-docx flask flask-cors pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.8 MB/s eta 0:00:00


In [ ]:
from docx import Document
from io import BytesIO
from docx.shared import Pt

def generate_docx(text: str):
    doc = Document()

    for line in text.split("\n"):
        p = doc.add_paragraph()
        run = p.add_run(line)
        run.font.name = "Times New Roman"
        run.font.size = Pt(12)

    buffer = BytesIO()
    doc.save(buffer)
    buffer.seek(0)

    return buffer


Explaining the Document

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.9 MB/s eta 0:00:00


In [ ]:
SYSTEM_MSG_EXP = """
Ești un asistent juridic specializat în analiza documentelor juridice în limba română.
Rolul tău este să explici clar și profesionist documente juridice existente,
fără a le modifica sau rescrie, folosind terminologie juridică corectă,
dar accesibilă unui client fără pregătire juridică.
"""
TEMPLATE_EXP = """
Analizează documentul juridic furnizat de client mai jos.

DOCUMENT:
\"\"\"
{document}
\"\"\"

Sarcini:

1. Rezumat:
- Oferă un rezumat clar și concis al documentului (maxim 4-5 rânduri),
- Evidențiază tipul documentului, părțile, scopul și obligațiile principale.

2. Explicație scurta:
- Explică pe înțelesul clientului fiecare secțiune importantă a documentului:
  - părțile contractante
  - obiectul
  - durata
  - obligațiile părților
  - preț/remunerație (dacă există)
  - clauze speciale (penalități, reziliere, confidențialitate etc.)
- Precizează efectele juridice principale pentru client.

3. Observații juridice:
- Semnalează eventuale clauze sensibile, neclare sau dezechilibrate
- Menționează riscuri potențiale pentru client
- Indică dacă lipsesc elemente esențiale din punct de vedere legal

4. Concluzie:
- Scurtă concluzie privind natura și impactul documentului

Nu rescrie și nu modifica documentul.
Nu adăuga clauze noi.
Nu oferi consultanță juridică personalizată.
"""
def build_prompt_analiza_contract(document):
    prompt = TEMPLATE_EXP.format(
        document=document,
    )
    return prompt



In [ ]:
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text


In [ ]:
def explain_document(pdf_path):
    text_pdf = extract_text_from_pdf(pdf_path)

    system_msg = SYSTEM_MSG_EXP
    prompt = build_prompt_analiza_contract(text_pdf)

    text = call_model(
        system_msg,
        prompt,
        temperature=0.4,
        max_new_tokens=500
    )

    text = clean_text(text, mode="text")
    return text


In [ ]:
!pip install rank-bm25

In [ ]:
import unicodedata

def normalize(text):
    """
    Transformă textul în lowercase și elimină diacritice
    """
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    return text



In [ ]:
!pip install faiss-cpu sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 41.8 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import faiss
import pickle
def load_all(folder="/content/rag_index"):
    # Texte
    with open(f"{folder}/chunk_texts (1).json", "r", encoding="utf-8") as f:
        chunk_texts = json.load(f)

    # Embeddings
    chunk_embeddings = np.load(f"{folder}/chunk_embeddings.npy")

    # FAISS
    index_cpu = faiss.read_index(f"{folder}/faiss_index.index")

    # BM25
    with open(f"{folder}/bm25_model.pkl", "rb") as f:
        bm25 = pickle.load(f)

    print(f"✅ Obiecte încărcate din {folder}")
    return chunk_texts, chunk_embeddings, index_cpu, bm25

In [ ]:
chunk_texts, chunk_embeddings, index_cpu, bm25 = load_all()

✅ Obiecte încărcate din /content/rag_index


In [ ]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
def hybrid_retrieve(query, top_k=5, keywords=None, alpha=0.5):
    """
    Hybrid search FAISS + BM25 + keyword filter cu normalizare diacritice

    alpha = pondere FAISS vs BM25 (0.5 = egal)
    """
    # --- Normalize query ---
    query_norm = normalize(query)

    # --- FAISS semantic search ---
    query_emb = embed_model.encode(query, convert_to_numpy=True)
    query_emb = query_emb / np.linalg.norm(query_emb)
    query_emb = query_emb.astype(np.float32).reshape(1, -1)

    D, I = index_cpu.search(query_emb, top_k*3)  # multiplicăm pentru filtrare
    faiss_scores = {idx: float(score) for idx, score in zip(I[0], D[0])}

    # --- BM25 keyword search ---
    tokenized_query = query_norm.split()
    bm25_scores_raw = bm25.get_scores(tokenized_query)
    bm25_scores = {i: score for i, score in enumerate(bm25_scores_raw)}

    # --- Combină scorurile ---
    combined_scores = {}
    for idx in range(len(chunk_texts)):
        s_faiss = faiss_scores.get(idx, 0)
        s_bm25 = bm25_scores.get(idx, 0)
        combined_scores[idx] = alpha*s_faiss + (1-alpha)*s_bm25

    # --- Sortează descrescător după scor ---
    top_indices = sorted(combined_scores, key=lambda i: combined_scores[i], reverse=True)

    # --- Aplica filtrarea keywords normalizate ---
    results = []
    keywords_norm = [normalize(kw) for kw in keywords] if keywords else []

    for idx in top_indices:
        text_norm = normalize(chunk_texts[idx])
        if keywords_norm:
            if not any(kw in text_norm for kw in keywords_norm):
                continue
        results.append({
            "index": idx + 1,
            "score": combined_scores[idx],
            "text": chunk_texts[idx]
        })
        if len(results) >= top_k:
            break

    return results



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "Care este procedura legală pentru reținerea salariului în cazul unor datorii ale salariatului către angajator?"
keywords = []

top_chunks = hybrid_retrieve(query, top_k=5, keywords=keywords, alpha=0.6)

for rank, chunk in enumerate(top_chunks, 1):
    print(f"[{rank}] Articol {chunk['index']} - Scor combinat ≈ {chunk['score']:.4f}")
    print(chunk['text'][:300] + "...\n")


[1] Articol 173 - Scor combinat ≈ 5.1863
Articolul 173         (1) Salariaţii beneficiază de protecţia drepturilor lor în cazul în care se produce un transfer al întreprinderii, al unităţii sau al unor părţi ale acesteia către un alt angajator, potrivit legii.   ...        (2) Drepturile şi obligaţiile cedentului, care decurg dintr-un cont...

[2] Articol 61 - Scor combinat ≈ 4.9007
Articolul 61     Angajatorul poate dispune concedierea pentru motive care ţin de persoana salariatului în următoarele situaţii:      a) în cazul în care salariatul a săvârşit o abatere gravă sau abateri repetate de la regulile de disciplină a muncii ori de la cele stabilite prin contractul individua...

[3] Articol 268 - Scor combinat ≈ 4.8113
Articolul 268         (1) Cererile în vederea soluţionării unui conflict de muncă pot fi formulate:   ...        a) în termen de 30 de zile calendaristice de la data în care a fost comunicată decizia unilaterală a angajatorului referitoare la încheierea, executarea, m

In [ ]:
# --- Funcția de generare răspuns ---
def generate_rag_answer(query, context_chunks, max_new_tokens=200):
    """
    Generează răspuns folosind modelul RoLlama și contextul extras.
    """
    # Extragem doar textul din chunk-uri
    context = "\n".join([chunk['text'] for chunk in context_chunks])

    prompt = (
        f"Răspunde la întrebarea următoare folosind doar contextul legal:\n"
        f"{context}\n"
        f"Întrebare: {query}\nRăspuns:"
    )
    print("Prompt pentru model:\n", prompt)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=max_new_tokens)

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    # Eliminăm prompt-ul dacă apare ca prefix
    if decoded.startswith(prompt):
        generated_only = decoded[len(prompt):].strip()
    else:
        generated_only = decoded.strip()

    generated_only = generated_only.replace("\\n", "\n")
    text=clean_text(generated_only)
    del inputs, output
    torch.cuda.empty_cache()
    return text

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("36cGVnOAUWw8s5S4l7miFjQdWXU_3qhXZYXWSyWh3EQufKqS1")


In [ ]:
!pip install flask pyngrok flask-cors


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
import threading
import tempfile
import os

# -----------------------------
# Flask App
# -----------------------------
app = Flask(__name__)
CORS(app)

# limită upload PDF (10 MB)
app.config["MAX_CONTENT_LENGTH"] = 10 * 1024 * 1024

@app.route("/")
def home():
    return "Server is running!"

# -----------------------------
# GENERATE DOCUMENT (JSON)
# -----------------------------
@app.route("/api/generate", methods=["POST"])
def api_generate():
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON payload"}), 400

    try:
        result_text = generate_text(data)
        buffer = generate_docx(result_text)
    except Exception as e:
        print("ERROR:", e)  # Vezi eroarea în Colab
        return jsonify({"error": str(e)}), 500

    return send_file(
        buffer,
        as_attachment=True,
        download_name="document.docx",
        mimetype="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
    )



# -----------------------------
# EXPLAIN DOCUMENT (PDF)
# -----------------------------
@app.route("/api/explain", methods=["POST"])
def api_explain_document():
    if "file" not in request.files:
        return jsonify({"error": "No file provided"}), 400

    file = request.files["file"]

    if file.filename == "":
        return jsonify({"error": "Empty filename"}), 400

    if not file.filename.lower().endswith(".pdf"):
        return jsonify({"error": "Only PDF files are allowed"}), 400

    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        file.save(tmp.name)
        pdf_path = tmp.name

    try:
        result = explain_document(pdf_path)
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally:
        os.remove(pdf_path)

    return jsonify({"response": result})

@app.route("/api/qa", methods=["POST"])
def api_qa():
    try:
        data = request.get_json()
        if not data or "question" not in data:
            return jsonify({"error": "Lipsește câmpul 'question'"}), 400

        question = data["question"]

        # 1. Recuperăm chunk-urile relevante (RAG)
        # presupunem că ai deja o funcție de retrieval
        keywords = []
        top_chunks = hybrid_retrieve(question, top_k=2, keywords=keywords, alpha=0.6)
        # context_chunks = [{"text": "..."}]

        if not top_chunks:
            return jsonify({
                "answer": "Nu am găsit informații relevante în contextul legal disponibil."
            })

        # 2. Generăm răspunsul cu LLM
        answer = generate_rag_answer(
            query=question,
            context_chunks=top_chunks
        )

        return jsonify({
            "answer": answer
        })

    except Exception as e:
        print("Eroare Q&A:", str(e))
        return jsonify({
            "error": "Eroare server la Q&A"
        }), 500

# -----------------------------
# Run Flask in background (DEV)
# -----------------------------
def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()

# -----------------------------
# ngrok tunnel
# -----------------------------
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Public URL: NgrokTunnel: "https://unilingual-tanisha-dissidently.ngrok-free.dev" -> "http://localhost:5000"


In [ ]:
import requests

url = "https://unilingual-tanisha-dissidently.ngrok-free.dev/api/generate"

data = {
    "TipCerere": "apel",
    "PartiImplicate": "persoană fizică",
    "Context": "constatare a unui fapt",
    "Conflict": "nu este cazul unui conflict juridic",
    "Argumente": "invoc prevederile legale aplicabile",
    "Termen": "în termen legal",
    "Ton": "strict formal"
}

response = requests.post(url, json=data)

if response.status_code == 200:
    with open("document.docx", "wb") as f:
        f.write(response.content)
    print("✅ Document salvat: document.docx")
else:
    print("❌ Eroare:", response.status_code, response.text)


entered build prompt

Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: apel
- Parti implicate: persoană fizică
- Context/Scop: constatare a unui fapt
- Conflict juridic: nu este cazul unui conflict juridic
- Argumente și probe: invoc prevederile legale aplicabile
- Termen dorit: în termen legal

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice. Adopta un ton strict formal, nu depasi 220 de cuvinte. Nu genera nimic in afara cererii.



INFO:werkzeug:127.0.0.1 - - [14/Jan/2026 06:15:08] "POST /api/generate HTTP/1.1" 200 -


✅ Document salvat: document.docx


Stop Server

In [ ]:
!fuser -k 5000/tcp